!pip install llama-index-tools-neo4j

In [ ]:
!pip install llama-index-readers-graphdb-cypher

In [108]:
import os
import json
import logging
import sys
from IPython.display import display, Markdown
import networkx as nx

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings, StorageContext, KnowledgeGraphIndex,download_loader, GPTVectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.tools.neo4j import Neo4jQueryToolSpec
from llama_index.agent.openai import OpenAIAgent
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.readers.graphdb_cypher import GraphDBCypherReader
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore

In [109]:
logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output, INFO for minimla output



Settings.llm = OpenAI(temperature=0, model="gpt-4", api_key=os.environ["OPENAI_API_KEY"])
Settings.chunk_size = 512
embed_dim = 1536

We start with the following base data model
<div>
<img src="base_data_model.png", width="300", align="left")
</div>

## Access Neo4j Credentials

In [110]:
# get Neo4j credentials (assume it's stored in credentials.json)
with open("credentials.json") as f:
    neo4j_connection_params = json.load(f)
    username = neo4j_connection_params["neo4j"]
    password = neo4j_connection_params["password"]
    url = neo4j_connection_params["url"]
    database = neo4j_connection_params["database"]
    os.environ["OPENAI_API_KEY"] = neo4j_connection_params['openai_api_key']

## Create Graph and Vector Stores

In [104]:
neo4j_vector = Neo4jVectorStore(username, password, url, embed_dim)
neo4j_graph_store = Neo4jGraphStore(username=username,password=password, url=url)

DEBUG:neo4j.pool:[#0000]  _: <POOL> created, routing address IPv4Address(('a9a7b965.databases.neo4j.io', 7687))
DEBUG:neo4j:[#0000]  _: <WORKSPACE> resolve home database
DEBUG:neo4j.pool:[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('a9a7b965.databases.neo4j.io', 7687))
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> in: a9a7b965.databases.neo4j.io:7687
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> dns resolver out: 52.88.247.84:7687
DEBUG:neo4j.pool:[#0000]  _: <POOL> _acquire router connection, database=None, address=ResolvedIPv4Address(('52.88.247.84', 7687))
DEBUG:neo4j.pool:[#0000]  _: <POOL> trying to hand out new connection
DEBUG:neo4j.io:[#0000]  C: <OPEN> 52.88.247.84:7687
DEBUG:neo4j.io:[#ECE3]  C: <SECURE> a9a7b965.databases.neo4j.io
DEBUG:neo4j.io:[#ECE3]  C: <MAGIC> 0x6060B017
DEBUG:neo4j.io:[#ECE3]  C: <HANDSHAKE> 0x00040405 0x00020404 0x00000104 0x00000003
DEBUG:neo4j.io:[#ECE3]  S: <HANDSHAKE> 0x00000405
DEBUG:neo4j.io:[#ECE3]  C: HELLO {'user_agent': 'neo4j-pytho

DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=257625.593, ttl=10, perf_time=257626.25 => False
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=False, has_server_for_mode=True, table routers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))} => True
DEBUG:neo4j.pool:[#0000]  _: <POOL> using existing routing table RoutingTable(database='neo4j' routers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))}, readers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))}, writers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))}, last_updated_time=257625.593, ttl=10)
DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire address, database='neo4j' address=IPv4Address(('a9a7b965.databases.neo4j.io', 7687))
DEBUG:neo4j.pool:[#ECE4]  _: <POOL> picked existing connection bolt-102864
DEBUG:neo4j.pool:[#ECE4]  _: <POOL> checked re_auth auth=None updated=False force=False
DEBUG:neo4j.pool:[#ECE4]  _: <POOL> handing out existing connectio

DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=None
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=257626.406, ttl=0, perf_time=257627.578 => False
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=neo4j
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=257627.031, ttl=10, perf_time=257627.593 => False
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=False, has_server_for_mode=True, table routers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))} => True
DEBUG:neo4j.pool:[#0000]  _: <POOL> using existing routing table RoutingTable(database='neo4j' routers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))}, readers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))}, writers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))}, last_updated_time=257627.031, ttl=10)
DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire address, database='neo4j' address=IPv4Address(('a9a7b965.databa

DEBUG:neo4j.io:[#ECE7]  S: RECORD * 1
DEBUG:neo4j.io:[#ECE7]  S: RECORD * 1
DEBUG:neo4j.io:[#ECE7]  S: RECORD * 1
DEBUG:neo4j.io:[#ECE7]  S: RECORD * 1
DEBUG:neo4j.io:[#ECE7]  S: SUCCESS {'bookmark': 'FB:kcwQYIbmufSsQpeqAuqnKeU98skFspA=', 'type': 'r', 't_last': 5, 'db': 'neo4j'}
DEBUG:neo4j.io:[#ECE7]  _: <CONNECTION> server state: STREAMING > READY
DEBUG:neo4j.pool:[#ECE7]  _: <POOL> released bolt-102868
DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=None
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=257626.406, ttl=0, perf_time=257628.687 => False
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=neo4j
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=257627.031, ttl=10, perf_time=257628.703 => False
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=False, has_server_for_m

## Create Chunks from Neo4j Nodes with Text Properties

In [111]:

query = """
    MATCH (d)
    WHERE (d.Text) is not null
    RETURN d"""
reader = GraphDBCypherReader(url, username, password, database)
documents = reader.load_data(query)

DEBUG:neo4j.pool:[#0000]  _: <POOL> created, routing address IPv4Address(('a9a7b965.databases.neo4j.io', 7687))
DEBUG:neo4j.pool:[#0000]  _: <POOL> close
DEBUG:neo4j.io:[#ECE8]  C: GOODBYE
DEBUG:neo4j.io:[#ECE8]  C: <CLOSE>
DEBUG:neo4j.io:[#ECE9]  C: GOODBYE
DEBUG:neo4j.io:[#ECE9]  C: <CLOSE>
DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=True, has_server_for_mode=False, table routers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))} => False
DEBUG:neo4j.pool:[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('a9a7b965.databases.neo4j.io', 7687))
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> in: a9a7b965.databases.neo4j.io:7687
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> dns resolver out: 54.245.8.251:7687
DEBUG:neo4j.pool:[#0000]  _: <POOL> _acquire router connection, database='neo4j', address=ResolvedIPv4Address(('54.245.8.251'

We data model now includes Entities and relationships between Entities
<div>
<img src="entity_data_model.png", width="300", align="left", padding="10px")
    </div>
<div>
<img src="entity_relationship_example.png", width="300", align="left")
     </div>

## The Storage Context is a connection from the DB (add in more here)

In [106]:
storage_context = StorageContext.from_defaults(graph_store=neo4j_graph_store)

## The following takes the documents and creates max triplets per chunk of 2 and writes back to the DB

Be sure to include the parameter '''include_embeddings=True''' so that the OpenAI embeddings are created in the DB.

In [107]:
new_index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
    include_embeddings=True,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/28 [00:00<?, ?it/s]

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "Executive Summary Based on much pre...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: As a second step, extensive data was collected\...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 1. Across all participating SU/SDs staff and\
 ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "A Citizen\u2019s Guide to  the CLA ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: CLA (common level of appraisal) is an adjustmen...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Here\u2019s how it works: 1. Town listers (or\
...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The CLA for that\
    \ town would be 80% or 0....
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: . . . $1,000 School tax rate . . . . . . . . . ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Why does the CLA seem to make scho

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Spaulding\
    \ High School, though, was forme...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "A History of the Individuals With D...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The law was last reauthorized in 2004, and the ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Many individuals lived in state institutions fo...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Many individuals lived in state institutions fo...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: For example,\
    \ the Pennsylvania Associatio...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: at that time.\n\nFacts and Figures\nIn the 1976...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: It also added\
    \ traumatic brain injury and...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: By the 1990-91 school year,\
    \

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: \n\nThe Vermont Agency of\
    \ Education expl...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "Rep. Sibilia: District forums on re...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: For grades 7-12, the parents or guardians may c...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Nevertheless,\
    \ every sect or denomination...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: No public dollars to religious\
    \ schools.\...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: \n\nTo be eligible for designation a school mus...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Authorizes an appeal to the State Board for agg...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  TRIGGER: The Joint Fiscal Office (JFO) was...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The intent\
    \ of this adequacy

Processing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

...

DEBUG:neo4j.io:[#ECE7]  S: SUCCESS {}
DEBUG:neo4j.io:[#ECE7]  S: SUCCESS {'t_first': 32, 'fields': []}
DEBUG:neo4j.io:[#ECE7]  _: <CONNECTION> server state: READY > STREAMING
DEBUG:neo4j.io:[#ECE7]  S: SUCCESS {'bookmark': 'FB:kcwQYIbmufSsQpeqAuqnKeU98skHHJA=', 'stats': {'contains-updates': True, 'labels-added': 1, 'relationships-created': 1, 'nodes-created': 1, 'properties-set': 1}, 'type': 'w', 't_last': 0, 'db': 'neo4j'}
DEBUG:neo4j.io:[#ECE7]  _: <CONNECTION> server state: STREAMING > READY
DEBUG:neo4j.pool:[#ECE7]  _: <POOL> released bolt-102868
DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire routing connection, access_mode='WRITE', database='neo4j'
DEBUG:neo4j.pool:[#0000]  _: <POOL> routing aged?, database=neo4j
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> purge check: last_updated_time=258152.296, ttl=10, perf_time=258159.359 => False
DEBUG:neo4j.pool:[#0000]  _: <ROUTING> checking table freshness (readonly=False): table expired=False, has_server_for_mode=True, table routers={IPv4Address

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x0000025F0C3453A0>, 'json_data': {'input': ["('Beck', 'Create', 'Incentive to spend less')", "('Beck', 'Aims', 'Revenue-neutral')"], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 14 Mar 2024 14:55:23 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'openai-mo

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x0000025F0C345DC0>, 'json_data': {'input': ["('Beck', 'Is putting into', 'Proposal')"], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 14 Mar 2024 14:55:25 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'openai-model', b'text-embedding-ada-002'), (b'openai-o

In [118]:
query_engine = new_index.as_query_engine(
    include_text=True, response_mode="tree_summarize"
)
response = query_engine.query(
    "Tell me about census-based funding?"
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nTell me about census-based funding?\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-4', 'stream': False, 'temperature': 0.0}}
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F08C32FA0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object 

DEBUG:neo4j.io:[#ED6D]  S: SUCCESS {}
DEBUG:neo4j.io:[#ED6D]  _: <CONNECTION> server state: AUTHENTICATION > READY
DEBUG:neo4j.io:[#ED6D]  C: TELEMETRY 2  # (<TelemetryAPI.AUTO_COMMIT: 2>)
DEBUG:neo4j.io:[#ED6D]  C: RUN 'MATCH p=(n1:Entity)-[*1..2]->() WHERE n1.id IN $subjs UNWIND relationships(p) AS rel WITH n1.id AS subj, p, apoc.coll.flatten(apoc.coll.toSet(collect([type(rel), endNode(rel).id]))) AS flattened_rels RETURN subj, collect(flattened_rels) AS flattened_rels LIMIT 30' {'subjs': ['funding']} {'db': 'neo4j'}
DEBUG:neo4j.io:[#ED6D]  _: <CONNECTION> client state: READY > STREAMING
DEBUG:neo4j.io:[#ED6D]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#ED6D]  S: SUCCESS {}
DEBUG:neo4j.io:[#ED6D]  S: SUCCESS {'t_first': 55, 'fields': ['subj', 'flattened_rels']}
DEBUG:neo4j.io:[#ED6D]  _: <CONNECTION> server state: READY > STREAMING
DEBUG:neo4j.io:[#ED6D]  S: SUCCESS {'bookmark': 'FB:kcwQYIbmufSsQpeqAuqnKeU98skHHpA=', 'type': 'r', 't_last': 1, 'db': 'neo4j'}
DEBUG:neo4j.io:[#ED6D]  _: <CONN

DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000025F00826640> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0C6EE0D0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 14 Mar 2024 15:09:03 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'openai-model', b'text-embedding-ada-002'), (b'openai-organization', b'neo4j-12'), (b'openai-proce

DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0A61DD00>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000025F0A5F1F40> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0A619880>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 14 Mar 2024 15:09:10 

In [119]:
response.response

'Census data plays a crucial role in determining funding for various educational and health programs. This includes special education, Head Start, school nutrition, after-school programming, classroom technology, as well as maternal and child health programs. The accuracy of the census count, particularly of young children, is of great importance as it directly influences the allocation of federal funds for these programs. Children under age 5 are the most likely to be underrepresented in the census, followed by children ages 5-9. An undercount in the census can have significant financial implications, as seen in the 2010 census where an estimated undercount of 1 to 2 million young children cost states collectively over $550 million a year for just five programs focused on young children. Therefore, many policymakers are focused on ensuring accurate census counts.'

## Let's look at the Vector Index

In [115]:
vector_storage_context = StorageContext.from_defaults(vector_store=neo4j_vector)
vector_index = GPTVectorStoreIndex.from_documents(
    documents, storage_context=vector_storage_context
)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "Executive Summary Based on much pre...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: As a second step, extensive data was collected\...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 1. Across all participating SU/SDs staff and\
 ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "A Citizen\u2019s Guide to  the CLA ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: CLA (common level of appraisal) is an adjustmen...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Here\u2019s how it works: 1. Town listers (or\
...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The CLA for that\
    \ town would be 80% or 0....
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: . . . $1,000 School tax rate . . . . . . . . . ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Why does the CLA seem to make scho

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Spaulding\
    \ High School, though, was forme...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "A History of the Individuals With D...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The law was last reauthorized in 2004, and the ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Many individuals lived in state institutions fo...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Many individuals lived in state institutions fo...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: For example,\
    \ the Pennsylvania Associatio...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: at that time.\n\nFacts and Figures\nIn the 1976...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: It also added\
    \ traumatic brain injury and...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: By the 1990-91 school year,\
    \

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: \n\nThe Vermont Agency of\
    \ Education expl...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "Rep. Sibilia: District forums on re...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: For grades 7-12, the parents or guardians may c...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Nevertheless,\
    \ every sect or denomination...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: No public dollars to religious\
    \ schools.\...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: \n\nTo be eligible for designation a school mus...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Authorizes an appeal to the State Board for agg...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  TRIGGER: The Joint Fiscal Office (JFO) was...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The intent\
    \ of this adequacy

DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0A1B0700>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000025F00826640> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0C314F40>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.comple

DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 14 Mar 2024 15:08:26 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'openai-model', b'text-embedding-ada-002'), (b'openai-organization', b'neo4j-12'), (b'openai-processing-ms', b'197'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'5000'), (b'x-ratelimit-limit-tokens', b'5000000'), (b'x-ratelimit-remaining-requests', b'4999'), (b'x-ratelimi

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



DEBUG:neo4j.io:[#ECE4]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#ECE4]  _: <CONNECTION> error: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
ERROR:neo4j.io:Failed to write data to connection IPv4Address(('a9a7b965.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('52.88.247.84', 7687)))
DEBUG:neo4j.io:[#ECE4]  C: <CLOSE>
DEBUG:neo4j.pool:[#0000]  _: <POOL> deactivating address IPv4Address(('a9a7b965.databases.neo4j.io', 7687))
DEBUG:neo4j.pool:[#0000]  _: <POOL> table={'neo4j': RoutingTable(database='neo4j' routers={}, readers={}, writers={}, last_updated_time=258945.906, ttl=10)}
DEBUG:neo4j.pool:[#ECE4]  _: <POOL> released bolt-102864


SessionExpired: Failed to write data to connection IPv4Address(('a9a7b965.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('52.88.247.84', 7687)))

In [120]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("What is census based funding?")

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x0000025F0A5D89D0>, 'json_data': {'input': ['What is census based funding?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0C311520>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000025F00826640> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F03079520>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:ht

DEBUG:neo4j.io:[#ED82]  _: <CONNECTION> client state: READY > STREAMING
DEBUG:neo4j.io:[#ED82]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#ED82]  S: SUCCESS {}
DEBUG:neo4j.io:[#ED82]  S: SUCCESS {'t_first': 23, 'fields': ['text', 'score', 'id', 'metadata']}
DEBUG:neo4j.io:[#ED82]  _: <CONNECTION> server state: READY > STREAMING
DEBUG:neo4j.io:[#ED82]  S: SUCCESS {'bookmark': 'FB:kcwQYIbmufSsQpeqAuqnKeU98skHHpA=', 'type': 'r', 't_last': 2, 'db': 'neo4j'}
DEBUG:neo4j.io:[#ED82]  _: <CONNECTION> server state: STREAMING > READY
DEBUG:neo4j.pool:[#ED82]  _: <POOL> released bolt-103566
DEBUG:llama_index.core.indices.utils:> Top 0 nodes:



In [121]:
print(response)

Empty Response


## Knowledge Graph RAG Retriever

In [127]:
graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=False,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
)

In [128]:
response = query_engine.query(
    "UVM",
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "A question is provided below. Given the question, extract up to 5 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\n---------------------\nUVM\n---------------------\nProvide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'\n"}], 'model': 'gpt-4', 'stream': False, 'temperature': 0.0}}
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F08242A00>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000025F0A5F1F40> server_hos

DEBUG:neo4j.io:[#ED6D]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#ED6D]  S: SUCCESS {}
DEBUG:neo4j.io:[#ED6D]  S: SUCCESS {'t_first': 59, 'fields': ['subj', 'flattened_rels']}
DEBUG:neo4j.io:[#ED6D]  _: <CONNECTION> server state: READY > STREAMING
DEBUG:neo4j.io:[#ED6D]  S: SUCCESS {'bookmark': 'FB:kcwQYIbmufSsQpeqAuqnKeU98skHHpA=', 'type': 'r', 't_last': 1, 'db': 'neo4j'}
DEBUG:neo4j.io:[#ED6D]  _: <CONNECTION> server state: STREAMING > READY
DEBUG:neo4j.pool:[#ED6D]  _: <POOL> released bolt-103558
DEBUG:llama_index.core.indices.knowledge_graph.retrievers:rel_map: {}
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No knowledge sequence extracted from entities.
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No knowledge sequence extracted from entities.


In [129]:
response

Response(response='Empty Response', source_nodes=[], metadata=None)

## The following queries the graph database with Cypher

In [122]:

llm = OpenAI(model="gpt-4", openai_api_key=os.environ["OPENAI_API_KEY"] , temperature=0)

gds_db = Neo4jQueryToolSpec(
    url=url,
    user=username,
    password=password,
    llm=llm,
    database=database,
)

tools = gds_db.to_tool_list()
agent = OpenAIAgent.from_tools(tools, verbose=True)

DEBUG:neo4j.pool:[#0000]  _: <POOL> created, routing address IPv4Address(('a9a7b965.databases.neo4j.io', 7687))
DEBUG:neo4j:[#0000]  _: <WORKSPACE> resolve home database
DEBUG:neo4j.pool:[#0000]  _: <POOL> attempting to update routing table from IPv4Address(('a9a7b965.databases.neo4j.io', 7687))
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> in: a9a7b965.databases.neo4j.io:7687
DEBUG:neo4j.io:[#0000]  _: <RESOLVE> dns resolver out: 52.88.247.84:7687
DEBUG:neo4j.pool:[#0000]  _: <POOL> _acquire router connection, database=None, address=ResolvedIPv4Address(('52.88.247.84', 7687))
DEBUG:neo4j.pool:[#0000]  _: <POOL> trying to hand out new connection
DEBUG:neo4j.io:[#0000]  C: <OPEN> 52.88.247.84:7687
DEBUG:neo4j.io:[#EDA9]  C: <SECURE> a9a7b965.databases.neo4j.io
DEBUG:neo4j.io:[#EDA9]  C: <MAGIC> 0x6060B017
DEBUG:neo4j.io:[#EDA9]  C: <HANDSHAKE> 0x00040405 0x00020404 0x00000104 0x00000003
DEBUG:neo4j.io:[#EDA9]  S: <HANDSHAKE> 0x00000405
DEBUG:neo4j.io:[#EDA9]  C: HELLO {'user_agent': 'neo4j-pytho

DEBUG:neo4j.io:[#EDAB]  S: SUCCESS {'server': 'Neo4j/5.18-aura', 'connection_id': 'bolt-103594', 'hints': {'connection.recv_timeout_seconds': 60, 'telemetry.enabled': True}}
DEBUG:neo4j.io:[#EDAB]  _: <CONNECTION> server state: CONNECTED > AUTHENTICATION
DEBUG:neo4j.io:[#EDAB]  S: SUCCESS {}
DEBUG:neo4j.io:[#EDAB]  _: <CONNECTION> server state: AUTHENTICATION > READY
DEBUG:neo4j.io:[#EDAB]  C: TELEMETRY 2  # (<TelemetryAPI.AUTO_COMMIT: 2>)
DEBUG:neo4j.io:[#EDAB]  C: RUN '\nCALL apoc.meta.data()\nYIELD label, other, elementType, type, property\nWHERE NOT type = "RELATIONSHIP" AND elementType = "node"\nWITH label AS nodeLabels, collect({property:property, type:type}) AS properties\nRETURN {labels: nodeLabels, properties: properties} AS output\n\n' {} {'db': 'neo4j'}
DEBUG:neo4j.io:[#EDAB]  _: <CONNECTION> client state: READY > STREAMING
DEBUG:neo4j.io:[#EDAB]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#EDAB]  S: SUCCESS {}
DEBUG:neo4j.io:[#EDAB]  S: SUCCESS {'t_first': 31, 'fields': ['output']

DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.io:[#EDAB]  S: RECORD * 1
DEBUG:neo4j.

DEBUG:neo4j.pool:[#0000]  _: <POOL> using existing routing table RoutingTable(database='neo4j' routers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))}, readers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))}, writers={IPv4Address(('a9a7b965.databases.neo4j.io', 7687))}, last_updated_time=259047.015, ttl=10)
DEBUG:neo4j.pool:[#0000]  _: <POOL> acquire address, database='neo4j' address=IPv4Address(('a9a7b965.databases.neo4j.io', 7687))
DEBUG:neo4j.pool:[#EDAB]  _: <POOL> picked existing connection bolt-103594
DEBUG:neo4j.pool:[#EDAB]  _: <POOL> checked re_auth auth=None updated=False force=False
DEBUG:neo4j.pool:[#EDAB]  _: <POOL> handing out existing connection
DEBUG:neo4j.io:[#EDAB]  C: TELEMETRY 2  # (<TelemetryAPI.AUTO_COMMIT: 2>)
DEBUG:neo4j.io:[#EDAB]  C: RUN '\n                CREATE CONSTRAINT IF NOT EXISTS FOR (n:Entity) REQUIRE n.id IS UNIQUE;\n                ' {} {'db': 'neo4j'}
DEBUG:neo4j.io:[#EDAB]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#EDAB]  S: SUCCESS {}
DEBUG:

In [141]:
output = agent.chat("Which people are associated with more than one organization and what are they?")

Added user message to memory: Which people are associated with more than one organization and what are they?
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Which people belong to more than one organization and which are they?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_qwZOJS2tQzdSgEnul5CWh0NQ', 'function': {'arguments': '{\n  "question": "Which people belong to more than one organization and which are they?"\n}', 'name': 'run_request'}, 'type': 'function'}]}, {'role': 'tool', 'content': "[['p.Person', 'organizations'], ['Jacqueline Kelleher', ['Vermont State Board of Education', 'U.S. DEPARTMENT OF EDUCATION']], ['William Mathis', ['Vermont State Board of Education', 'National Education Policy Center']], ['Kieran Killeen', ['UVM', 'University of Vermont']], ['Michael Steven Martin', ['University of Vermont', 'Rowland Foundation']]]", 'name': 'run_

DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0A62BA90>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000025F7FC88BC0> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0A625D60>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.comple

DEBUG:neo4j.io:[#C391]  C: <SECURE> a9a7b965.databases.neo4j.io
DEBUG:neo4j.io:[#C391]  C: <MAGIC> 0x6060B017
DEBUG:neo4j.io:[#C391]  C: <HANDSHAKE> 0x00040405 0x00020404 0x00000104 0x00000003
DEBUG:neo4j.io:[#C391]  S: <HANDSHAKE> 0x00000405
DEBUG:neo4j.io:[#C391]  C: HELLO {'user_agent': 'neo4j-python/5.16.0 Python/3.9.12-final-0 (win32)', 'routing': {'address': 'a9a7b965.databases.neo4j.io:7687'}, 'bolt_agent': {'product': 'neo4j-python/5.16.0', 'platform': 'Windows 10; AMD64', 'language': 'Python/3.9.12-final-0', 'language_details': 'CPython; 3.9.12-final-0 (main, Apr  4 2022 05:22:27) [MSC v.1916 64 bit (AMD64)]'}}
DEBUG:neo4j.io:[#C391]  _: <CONNECTION> client state: CONNECTED > AUTHENTICATION
DEBUG:neo4j.io:[#C391]  C: LOGON {'scheme': 'basic', 'principal': 'neo4j', 'credentials': '*******'}
DEBUG:neo4j.io:[#C391]  _: <CONNECTION> client state: AUTHENTICATION > READY
DEBUG:neo4j.io:[#C391]  S: SUCCESS {'server': 'Neo4j/5.18-aura', 'connection_id': 'bolt-115473', 'hints': {'conne

DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0A626760>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000025F0A5F1F40> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0A626730>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 14 Mar 2024 21:46:21 

In [142]:
display(Markdown(f"<b>{output.response}</b>"))

<b>Here are the people who are associated with more than one organization:

1. Jacqueline Kelleher is associated with the Vermont State Board of Education and the U.S. Department of Education.
2. William Mathis is associated with the Vermont State Board of Education and the National Education Policy Center.
3. Kieran Killeen is associated with UVM and the University of Vermont.
4. Michael Steven Martin is associated with the University of Vermont and the Rowland Foundation.</b>

In [134]:
from llama_index.core import VectorStoreIndex

In [135]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(vector_store=neo4j_vector)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "Executive Summary Based on much pre...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: As a second step, extensive data was collected\...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 1. Across all participating SU/SDs staff and\
 ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "A Citizen\u2019s Guide to  the CLA ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: CLA (common level of appraisal) is an adjustmen...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Here\u2019s how it works: 1. Town listers (or\
...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The CLA for that\
    \ town would be 80% or 0....
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: . . . $1,000 School tax rate . . . . . . . . . ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Why does the CLA seem to make scho

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Spaulding\
    \ High School, though, was forme...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "A History of the Individuals With D...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The law was last reauthorized in 2004, and the ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Many individuals lived in state institutions fo...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Many individuals lived in state institutions fo...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: For example,\
    \ the Pennsylvania Associatio...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: at that time.\n\nFacts and Figures\nIn the 1976...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: It also added\
    \ traumatic brain injury and...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: By the 1990-91 school year,\
    \

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: \n\nThe Vermont Agency of\
    \ Education expl...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  Text: "Rep. Sibilia: District forums on re...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: For grades 7-12, the parents or guardians may c...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Nevertheless,\
    \ every sect or denomination...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: No public dollars to religious\
    \ schools.\...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: \n\nTo be eligible for designation a school mus...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Authorizes an appeal to the State Board for agg...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: d:
  TRIGGER: The Joint Fiscal Office (JFO) was...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The intent\
    \ of this adequacy

DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F03079520>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000025F00826640> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0A60AD90>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.comple

DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 14 Mar 2024 16:56:29 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'openai-model', b'text-embedding-ada-002'), (b'openai-organization', b'neo4j-12'), (b'openai-processing-ms', b'181'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'5000'), (b'x-ratelimit-limit-tokens', b'5000000'), (b'x-ratelimit-remaining-requests', b'4999'), (b'x-ratelimi

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



DEBUG:neo4j.io:[#F198]  _: <CONNECTION> client state: READY > STREAMING
DEBUG:neo4j.io:[#F198]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#F198]  S: SUCCESS {}
DEBUG:neo4j.io:[#F198]  S: SUCCESS {'t_first': 192, 'fields': []}
DEBUG:neo4j.io:[#F198]  _: <CONNECTION> server state: READY > STREAMING
DEBUG:neo4j.io:[#F198]  S: SUCCESS {'bookmark': 'FB:kcwQYIbmufSsQpeqAuqnKeU98skHH5A=', 'stats': {'contains-updates': True, 'labels-added': 172, 'nodes-created': 172, 'properties-set': 860}, 'type': 'w', 't_last': 0, 'db': 'neo4j', 'notifications': [{'severity': 'WARNING', 'description': "The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')", 'code': 'Neo.ClientNotification.Statement.FeatureDeprecationWarning', 'position': {'column': 80, 'offset': 79, 'line': 1}, 'title': 'This feature is deprecated and will be removed in future versions.', 'category': 'DEPRECATION'}]}
DEBUG:neo4j.io:[#F198]  _: <CONNECTION> server state: STREAM

In [140]:
query_engine = index.as_query_engine()
response = query_engine.query("Tell me about the University of Vermont?")
display(Markdown(f"<b>{response}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x0000025F0A623D30>, 'json_data': {'input': ['Tell me about the University of Vermont?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0DFD9A60>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000025F00826640> server_hostname='api.openai.com' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000025F0A01A730>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST'

DEBUG:neo4j.io:[#C334]  C: PULL {'n': 1000}
DEBUG:neo4j.io:[#C334]  S: SUCCESS {}
DEBUG:neo4j.io:[#C334]  S: SUCCESS {'t_first': 2, 'fields': ['text', 'score', 'id', 'metadata']}
DEBUG:neo4j.io:[#C334]  _: <CONNECTION> server state: READY > STREAMING
DEBUG:neo4j.io:[#C334]  S: RECORD * 1
DEBUG:neo4j.io:[#C334]  S: RECORD * 1
DEBUG:neo4j.io:[#C334]  S: SUCCESS {'bookmark': 'FB:kcwQYIbmufSsQpeqAuqnKeU98skHH5A=', 'type': 'r', 't_last': 2, 'db': 'neo4j'}
DEBUG:neo4j.io:[#C334]  _: <CONNECTION> server state: STREAMING > READY
DEBUG:neo4j.pool:[#C334]  _: <POOL> released bolt-115436
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 54bd680a-0dc7-40ce-b9d2-780357ceb2fa] [Similarity score:             0.911102] d:
  Text: "Commentary\nLegislative update: Preserving Vermont's unique education landscape\n\
  ...
> [Node da86eb45-1bd3-4a1a-9282-8fd42f6d4fca] [Similarity score:             0.905454] \xA7\xA7 821-36\n\n(Last updated December 14, 2023) \n\nLegal History\n\
    On March 17,

DEBUG:httpcore.http11:response_closed.complete
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"


<b>The context does not provide any information about the University of Vermont.</b>